In [ ]:
#原网址  https://www.jianshu.com/p/f931a4df202c

In [1]:
import pandas as pd
from sklearn.ensemble import *
import sklearn.metrics as sm

In [2]:
import numpy as np
np.set_printoptions(suppress=True,threshold=np.nan)

In [3]:
pd.option_context('display.precision', 3)

In [4]:
path='E:/学习/python/基于python的信用评分卡模型/cs-training.csv'   #路径中出现中文名时需要以此方式打开
f=open(path,encoding='utf-8')
data=pd.read_csv(f)

In [5]:
data=data.drop(['Unnamed: 0'],axis=1)
data.describe()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,1.202690e+05,150000.000000,150000.000000,150000.000000,150000.000000,146076.000000
mean,0.066840,6.048438,52.295207,0.421033,353.005076,6.670221e+03,8.452760,0.265973,1.018240,0.240387,0.757222
std,0.249746,249.755371,14.771866,4.192781,2037.818523,1.438467e+04,5.145951,4.169304,1.129771,4.155179,1.115086
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.029867,41.000000,0.000000,0.175074,3.400000e+03,5.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.154181,52.000000,0.000000,0.366508,5.400000e+03,8.000000,0.000000,1.000000,0.000000,0.000000
75%,0.000000,0.559046,63.000000,0.000000,0.868254,8.249000e+03,11.000000,0.000000,2.000000,0.000000,1.000000
max,1.000000,50708.000000,109.000000,98.000000,329664.000000,3.008750e+06,58.000000,98.000000,54.000000,98.000000,20.000000


In [6]:
data.shape

(150000, 11)

## 缺失值处理

In [12]:
#原始数据需要去掉第一列
#用随机森林对缺失值预测填充函数
def set_missing(df):
    #把已有的数值型特征取出来
    process_df=df.ix[:,[5,0,1,2,3,4,6,7,8,9]]
    #分成已知该特征和未知该特征两部分
    known=process_df[process_df.MonthlyIncome.notnull()].as_matrix()
    unknown=process_df[process_df.MonthlyIncome.isnull()].as_matrix()
    #X为特征属性值
    X=known[:,1:]
    #y为结果标签
    y=known[:,0]
    #fit到randomforestregressor之中
    rfr=RandomForestRegressor(random_state=0,n_estimators=200,max_depth=3,n_jobs=-1)
    rfr.fit(X,y)
    #用得到的模型进行未知特征值预测
    predicted=rfr.predict(unknown[:,1:]).round(0)
    print(predicted)
    df.loc[(df.MonthlyIncome.isnull()),'MonthlyIncome']=predicted
    return df

In [13]:
data=set_missing(data)#用随机森林填补比较多的缺失值
data=data.dropna()#删除比较少的缺失值
data=data.drop_duplicates()#删除重复项
#data.to_csv('MissingData.csv',index=False)

D:\mathwork\Anaconda\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
D:\mathwork\Anaconda\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
D:\mathwork\Anaconda\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


[8311. 1159. 8311. 2554. 1159. 5356. 2554. 8311. 8311. 8311. 8311. 1159.
 1159. 8311. 1159. 1159. 8311. 2554. 2554. 1159. 1159. 2554. 2554. 1159.
 2554. 2554. 2554. 1159. 2554. 1159. 2554. 8311. 8311. 2554. 2554. 4826.
 8311. 1159. 2554. 2554. 8311. 2554. 1159. 8311. 4826. 2554. 1159. 8311.
 1159. 8311. 2554. 2554. 8311. 8311. 8311. 2554. 1159. 1159. 2554. 2554.
 1159. 5356. 1159. 8311. 2554. 2554. 2554. 1159. 1159. 2554. 2554. 2554.
 2554. 1159. 2554. 1159. 5356. 2554. 1159. 2554. 5356. 8311. 8311. 8311.
 1159. 1159. 1159. 1159. 1159. 1159. 8311. 2554. 2554. 1159. 2554. 1159.
 1159. 2554. 2554. 8311. 2554. 8311. 8311. 1159. 8311. 2554. 1159. 1159.
 8311. 1159. 2554. 1159. 2554. 1159. 8311. 2554. 2554. 1159. 1159. 8311.
 2554. 1159. 2554. 2554. 8311. 1159. 2554. 1159. 1159. 1159. 1159. 2554.
 1159. 1159. 5356. 1159. 1159. 1159. 2554. 1159. 2554. 8311. 1159. 2554.
 1159. 1159. 8311. 2554. 2554. 2554. 2554. 2554. 2554. 8311. 1159. 8311.
 1159. 2554. 2554. 8311. 2554. 1159. 8311. 2554. 83

## 异常值处理

In [14]:
#年龄扥等于0的剔除
data=data[data['age']>0]
#剔除异常值
data=data[data['NumberOfTime30-59DaysPastDueNotWorse']<90]
#原数据集中好客户为0，违约客户为1，考虑到正常的理解，能正常履约并支付利息的客户为1，所以对数据取反
data['SeriousDlqin2yrs']=1-data['SeriousDlqin2yrs']

## 数据切分

In [16]:
from sklearn.cross_validation import train_test_split
Y=data['SeriousDlqin2yrs']
X=data.ix[:,1:]
#测试机占比30%
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=0)
#print(Y_train)
train=pd.concat([Y_train,X_train],axis=1)
test=pd.concat([Y_test,X_test],axis=1)
clasTest=test.groupby('SeriousDlqin2yrs')['SeriousDlqin2yrs'].count()
#train.to_csv('TrainData.csv',index=False)
#test.to_csv('TestData.csv',index=False)

D:\mathwork\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\mathwork\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


## 探索性分析

## 变量选择

### 分箱处理

In [17]:
#变量分箱（binning）是对连续变量离散化（discretization）的一种称呼。信用评分卡开发中一般
#有常用的等距分段、等深分段、最优分段。其中等距分段（Equval length intervals）是指分段的
#区间是一致的，比如年龄以十年作为一个分段；等深分段（Equal frequency intervals）是先确定
#分段数量，然后令每个分段中数据数量大致相等；最优分段（Optimal Binning）又叫监督离散化
#（supervised discretizaion），使用递归划分（Recursive Partitioning）将连续变量分为分段，
#背后是一种基于条件推断查找较佳分组的算法。
#首先对连续变量考虑进行最优分段，在连续变量的分布不满足最优分段的要求时，在考虑对连续变量进行
#等距分段，最优分箱的代码如下：
def mono_bin(Y,X,n=20):
    r=0
    good=Y.sum()
    bad=Y.count()-good
    while np.abs(r)<1:
        d1=pd.DataFrame({'X':X,"Y":Y,"Bucket":pd.qcut(X,n)})
        d2=d1.groupby('Bucket',as_index=True)
        r,q=stats.sepearmanr(d2.mean().X,d2.mean().Y)
        n=n-1
    d3=pd.DataFrame(d2.X.min(),columns=['min'])
    d3['min']=d2.min().X
    d3['max']=d2.max().X
    d3['sum']=d2.sum().Y
    d3['total']=d2.count().Y
    d3['rate']=d2.mean().Y
    d3['woe']=np.log((d3['rate']/(1-d3['rate']))/(good/bad))
    d3['goodattribute']=d3['sum']/good
    d3['badattribute']=(d3['total']-d3['sum'])/bad
    iv=((d3['goodattribute']-d3['badattribute'])*d3['woe']).sum()
    d4=(d3.sort_index(by='min')).reset_index(drop=True)
    print("="*60)
    print(d4)
    cut=[]
    cut.append(float('-inf'))
    for i in range(1,n+1):
        qua=X.quantile(i/(n+1))
        cut.append(round(qua,4))
    cut.append(float('inf'))
    woe=list(d4['woe'].round(3))
    return d4,iv,cut,woe



In [18]:
#针对不能最优分箱的变量，分箱如下
#连续变量离散化
cutx3=[ninf,0,1,3,5,pinf]
cutx6=[ninf,1,2,3,5,pinf]
cutx7=[ninf,0,1,3,5,pinf]
cutx8=[ninf,0,1,2,3,pinf]
cutx9=[ninf,0,1,3,pinf]
cutx10=[ninf,0,1,2,3,5,pinf]

NameError: name 'ninf' is not defined

### woe

### 相关性分析和IV筛选

In [ ]:
corr=data.corr()#计算各变量的相关性系数
xticks=['x0','x1','x2','x3','x4','x5','x6','x7','x8','x9','x10']#x轴标签
yticks=list(corr.index)#y轴标签
fig=plt.figure()
ax1=fig.add_subplot(1,1,1)
sns.heatmap(corr,annot=True,cmap='rainbow',ax=ax1,annot_kws={'size': 9, 'weight': 'bold', 'color': 'blue'})#绘制相关性系数热力图
ax1.set_xticklabels(xticks,rotation=0,fontsize=10)
ax1.set_yticklabels(yticks,rotation=0,fontsize=10)
plt.show()

In [ ]:
#IV判断标准
#< 0.02: unpredictive
#0.02 to 0.1: weak
#0.1 to 0.3: medium
#0.3 to 0.5: strong
#> 0.5: suspicious
#生成IV图
ivlist=[ivx1,ivx2,ivx3,ivx4,ivx5,ivx6,ivx7,ivx8,ivx9,ivx10]#各变量IV
index=['x0','x1','x2','x3','x4','x5','x6','x7','x8','x9','x10']
fig1=plt.figure(1)
ax1=fig1.add_subplot(1,1,1)
x=np.arange(len(index))+1
ax1.bar(x,ivlist,width=0.4)#生成柱状图
ax1.set_xticks(x)
ax1.set_xticklabels(index, rotation=0, fontsize=12)
ax1.set_ylabel('IV(Information Value)', fontsize=14)
#在柱状图上添加数字标签
for a,b in zip(x,ivlist):
    plt.text(a,b+0.01,'%.4f'% b,ha='center',va='bottom',fontsize=10)
plt.show()


## woe转换

In [ ]:
#在建立模型之前，我们需要将筛选后的变量转换为WoE值，便于信用评分
#替换成woe函数
def replace_woe(series,cut,woe):
    list=[]
    i=0
    while i<len(series):
        value=series[i]
        j=len(cut)-2
        m=len(cut)-2
        while j>=0:
            if value>=cut[j]:
                j=-1
            else:
                j-=1
                m-=1
            list.append(woe[m])
            i+=1
    return list

In [ ]:
#将每个变量都进行替换，并将其保存到woedata文件中
#替换成woe
data['RevolvingUtilizationOfUnsecuredLines']=Series(replace_woe(data['RevolvingUtilizationOfUnsecuredLines'],cutx1,woex1))
data['age']=Series(replace_woe(data['age'],cutx2,woex2))
data['NumberOfTime30-59DaysPastDueNotWorse'] = Series(replace_woe(data['NumberOfTime30-59DaysPastDueNotWorse'], cutx3, woex3))
data['DebtRatio'] = Series(replace_woe(data['DebtRatio'], cutx4, woex4))
data['MonthlyIncome'] = Series(replace_woe(data['MonthlyIncome'], cutx5, woex5))
data['NumberOfOpenCreditLinesAndLoans'] = Series(replace_woe(data['NumberOfOpenCreditLinesAndLoans'], cutx6, woex6))
data['NumberOfTimes90DaysLate'] = Series(replace_woe(data['NumberOfTimes90DaysLate'], cutx7, woex7))
data['NumberRealEstateLoansOrLines'] = Series(replace_woe(data['NumberRealEstateLoansOrLines'], cutx8, woex8))
data['NumberOfTime60-89DaysPastDueNotWorse'] = Series(replace_woe(data['NumberOfTime60-89DaysPastDueNotWorse'], cutx9, woex9))
data['NumberOfDependents'] = Series(replace_woe(data['NumberOfDependents'], cutx10, woex10))
#~~data.to_csv('WoeData.csv', index=False)

## logisic模型建立

In [ ]:
#~~data=pd.read_csv('Woedata.csv')
#因变量
Y=data['SeriousDlqin2yrs']
#自变量，剔除对因变量影响不明显的变liang
X=data.drop(['SeriousDlqin2yrs','DebtRatio','MonthlyIncome','NumberOfOpenCreditLinesAndLoans','NumberRealEstateLoansOrLines','NumberOfDependents'],axis=1)
X1=sm.add_constant(X)
logit=sm.Logit(Y,X1)
result=logit.fit()
print(result.summary())


### 模型检验

In [ ]:
#因变量
Y_test=test['SeriousDlqin2yrs']
#自变量。剔除因变量影响不明显的变量，与模型变量对应
X_test=test.drop(['SeriousDlqin2yrs', 'DebtRatio', 'MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans','NumberRealEstateLoansOrLines', 'NumberOfDependents'],axis=1)
X3=sm.add_constant(X_test)
resu=result.predict(X3)#进行预测
fpr,tpr,threshold=roc_curve(Y_test,resu)
rocauc=auc(fpr,tpr)#计算AUC
plt.plot(fpr,tpr,'b',label='AUC=%0.2f'% rocauc)#生成roc曲线
plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('真正率')
plt.xlabel('假正率')
plt.show()

## 信用评分

In [ ]:
# 我们取600分为基础分值，PDO为20（每高20分好坏比翻一倍），好坏比取20。
p = 20 / math.log(2)
q = 600 - 20 * math.log(20) / math.log(2)
baseScore = round(q + p * coe[0], 0)


In [ ]:
#计算分数函数
def get_score(coe,woe,factor):
    scores=[]
    for w in woe:
        score=round(coe*w*factor,0)
        scores.append(score)
    return scores

In [ ]:
# 各项部分分数
x1 = get_score(coe[1], woex1, p)
x2 = get_score(coe[2], woex2, p)
x3 = get_score(coe[3], woex3, p)
x7 = get_score(coe[4], woex7, p)
x9 = get_score(coe[5], woex9, p)


In [ ]:
#根据变量计算分数
def compute_score(series,cut,score):
    list = []
    i = 0
    while i < len(series):
        value = series[i]
        j = len(cut) - 2
        m = len(cut) - 2
        while j >= 0:
            if value >= cut[j]:
                j = -1
            else:
                j -= 1
                m -= 1
        list.append(score[m])
        i += 1
    return list


In [ ]:
test1 = pd.read_csv('TestData.csv')
test1['BaseScore']=Series(np.zeros(len(test1)))+baseScore
test1['x1'] = Series(compute_score(test1['RevolvingUtilizationOfUnsecuredLines'], cutx1, x1))
test1['x2'] = Series(compute_score(test1['age'], cutx2, x2))
test1['x3'] = Series(compute_score(test1['NumberOfTime30-59DaysPastDueNotWorse'], cutx3, x3))
test1['x7'] = Series(compute_score(test1['NumberOfTimes90DaysLate'], cutx7, x7))
test1['x9'] = Series(compute_score(test1['NumberOfTime60-89DaysPastDueNotWorse'], cutx9, x9))
test1['Score'] = test1['x1'] + test1['x2'] + test1['x3'] + test1['x7'] +test1['x9']  + baseScore
test1.to_csv('ScoreData.csv', index=False)
